## ひろしまQuest2020#stayhome【球種予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_9.f'
SUBMIT = 'submit/lgbm_9.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 151)

### 不要な列を削除

In [4]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [5]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [6]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 136)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,batter_cnt_bat,bat_game_cnt_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,foreigner,batter_cnt,bat_game_cnt,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 136)

### test
- 行数: 521650

In [8]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 136)

In [9]:
del all_pitch
gc.collect()

60

In [10]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [11]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [12]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [13]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 1.15235 + 0.00190392
[200]	cv_agg's multi_logloss: 1.13182 + 0.00180466
[300]	cv_agg's multi_logloss: 1.12472 + 0.00182401
[400]	cv_agg's multi_logloss: 1.1206 + 0.00188234
[500]	cv_agg's multi_logloss: 1.11751 + 0.00201057
[600]	cv_agg's multi_logloss: 1.11536 + 0.00216701
[700]	cv_agg's multi_logloss: 1.11375 + 0.00227534
[800]	cv_agg's multi_logloss: 1.11268 + 0.00228997
[900]	cv_agg's multi_logloss: 1.11196 + 0.0023922
[1000]	cv_agg's multi_logloss: 1.11157 + 0.00250631
[1100]	cv_agg's multi_logloss: 1.11134 + 0.00255299
[1200]	cv_agg's multi_logloss: 1.11138 + 0.00257056
Best num_boost_round: 1106
Best CV score: 1.1113177029272225
CPU times: user 1h 26min 32s, sys: 1min 36s, total: 1h 28min 8s
Wall time: 25min 44s


In [14]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [15]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 25min 59s, sys: 18.7 s, total: 26min 17s
Wall time: 7min 5s


### Feature Importance

In [16]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
129,age_dif_b-c,2026
64,fork_pit,2081
14,ball_cnt,2091
85,weight_bat,2170
84,height_bat,2183
61,curve_pit,2211
119,salary_x_year_dif_p-b,2269
96,salary_year_bat,2316
6,player_cnt_in_game,2334
127,salary_dif_b-c,2358


In [17]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 14min 59s, sys: 4.78 s, total: 15min 4s
Wall time: 4min 8s


## 結果出力

In [18]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.372151,0.026985,0.124704,0.323536,0.002590,0.000027,0.000023,0.149984
1,1,0.317922,0.059396,0.118809,0.416006,0.004341,0.000025,0.000015,0.083486
2,2,0.291704,0.076478,0.141924,0.264097,0.105609,0.000130,0.000029,0.120029
3,3,0.324008,0.097417,0.167235,0.320762,0.016699,0.000059,0.000018,0.073801
4,4,0.229526,0.037944,0.285210,0.336024,0.013350,0.000066,0.000022,0.097858


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [19]:
submit.mean()

index    260824.500000
0             0.558694
1             0.062969
2             0.169841
3             0.048420
4             0.072505
5             0.039607
6             0.006141
7             0.041823
dtype: float64

In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [21]:
'signate submit --competition-id=275 ~/work/signate/baseball/{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=275 ~/work/signate/baseball/submit/lgbm_9.csv --note feat=136_cv=1.1113177029272225'

### 評価結果
#### 9 (136) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806